## GPT 5 Vision을 활용한 이미지 처리

### 순서
1. 이미지 분석 및 마케팅 아이디어, 짧은 스토리 생성
2. 차트/그래프 데이터 분석

In [ ]:
## 라이브러리 설치
%pip install openai pillow requests matplotlib dotenv

In [2]:
from dotenv import load_dotenv
import openai
import base64
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
from datetime import datetime

load_dotenv()

# OpenAI 클라이언트 초기화
client = openai.OpenAI()

In [3]:
plt.rc('font', family='Malgun Gothic')

In [4]:
# helper functions

def encode_image_to_base64(image_path):
    """
    이미지 파일을 base64로 인코딩하는 함수
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image_with_gpt5(image_path, prompt, max_tokens=300):
    """
    GPT-5 API를 사용하여 이미지를 분석하는 함수
    """
    # 이미지를 base64로 인코딩
    base64_image = encode_image_to_base64(image_path)
    
    try:
        response = client.chat.completions.create(
            model="gpt-5", 
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/webp;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_completion_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

def display_image(image_path):
    """
    이미지를 화면에 표시하는 함수
    """
    img = Image.open(image_path)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"분석 대상 이미지: {os.path.basename(image_path)}")
    plt.show()



### 예제 1: 이미지 분석 및 마케팅 아이디어, 짧은 스토리 생성

- 이미지의 내용, 객체, 색상, 분위기 등을 분석
- 마케팅 카피 생성
- 이미지 기반 스토리텔링

In [5]:
# 카페 내부 이미지 다운로드 후 경로 저장
image_path = '6_MultiModal\img'

In [ ]:
# 이미지 표시
display_image(image_path)

# 다양한 프롬프트로 이미지 분석
prompts = [
    "이 이미지를 자세히 분석하고 설명해주세요. 무엇이 보이고, 어떤 분위기인지 알려주세요.",
    
    "이 이미지를 보고 카페 마케팅 문구를 3개 만들어주세요. 감성적이고 매력적인 문구로 작성해주세요.",
    
    "이 장소에서 일어날 수 있는 짧은 스토리를 한국어로 작성해주세요. 등장인물과 상황을 포함해서요."
]

print("=" * 60)
print("🎯 GPT 5 이미지 분석 결과")
print("=" * 60)

for i, prompt in enumerate(prompts, 1):
    print(f"\n📝 분석 {i}: {prompt[:30]}...")
    print("-" * 50)
    
    # GPT 5로 이미지 분석
    result = analyze_image_with_gpt5(image_path, prompt, max_tokens=500)
    
    print(f"💡 결과:\n{result}")
    print("-" * 50)


### 예제 2: 차트/그래프 데이터 분석

- 차트의 데이터 포인트 읽기
- 트렌드 분석 및 해석
- 비즈니스 인사이트 도출
- 개선 방안 제안


In [ ]:
# 샘플 비즈니스 데이터 차트 생성
import numpy as np

months = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']
revenue_2023 = [850, 920, 1150, 1080, 1200, 1350, 1500, 1420, 1300, 1450, 1600, 1800]
revenue_2024 = [950, 1020, 1250, 1180, 1400, 1550, 1650, 1520, 1480, 1650, 1750, 1950]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('2023-2024 비즈니스 성과 대시보드', fontsize=16, fontweight='bold')

# 1. 라인 차트 - 월별 매출 추이
ax1.plot(months, revenue_2023, marker='o', linewidth=2, label='2023년', color='#FF6B6B')
ax1.plot(months, revenue_2024, marker='s', linewidth=2, label='2024년', color='#4ECDC4')
ax1.set_title('월별 매출 추이 (만원)', fontsize=12, fontweight='bold')
ax1.set_ylabel('매출 (만원)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 2. 바 차트 - 분기별 매출 비교
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
q_2023 = [np.mean(revenue_2023[i:i+3]) for i in range(0, 12, 3)]
q_2024 = [np.mean(revenue_2024[i:i+3]) for i in range(0, 12, 3)]

x = np.arange(len(quarters))
width = 0.35
ax2.bar(x - width/2, q_2023, width, label='2023년', color='#FF6B6B', alpha=0.8)
ax2.bar(x + width/2, q_2024, width, label='2024년', color='#4ECDC4', alpha=0.8)
ax2.set_title('분기별 평균 매출 비교', fontsize=12, fontweight='bold')
ax2.set_ylabel('매출 (만원)')
ax2.set_xticks(x)
ax2.set_xticklabels(quarters)
ax2.legend()

# 3. 파이 차트 - 2024년 분기별 매출 비중
ax3.pie(q_2024, labels=quarters, autopct='%1.1f%%', startangle=90, 
        colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax3.set_title('2024년 분기별 매출 비중', fontsize=12, fontweight='bold')

# 4. 성장률 차트
growth_rate = [(r24/r23 - 1) * 100 for r23, r24 in zip(revenue_2023, revenue_2024)]
colors = ['#FF6B6B' if gr < 10 else '#4ECDC4' if gr < 20 else '#45B7D1' for gr in growth_rate]
ax4.bar(months, growth_rate, color=colors, alpha=0.8)
ax4.set_title('전년 대비 월별 성장률 (%)', fontsize=12, fontweight='bold')
ax4.set_ylabel('성장률 (%)')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

ax4.text(0.02, 0.98, '색상별 성장률:\n🔴 < 10%\n🟢 10-20%\n🔵 > 20%', 
         transform=ax4.transAxes, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()

# 차트 저장
chart_image_path = './business_dashboard.jpg'
plt.savefig(chart_image_path, dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# 대시보드 분석

display_image(chart_image_path)

# 비즈니스 분석을 위한 프롬프트
business_prompts = [
    """
    이 비즈니스 대시보드를 자세히 분석해주세요. 
    각 차트에서 보이는 주요 수치와 트렌드를 읽어주시고, 
    전반적인 비즈니스 성과를 평가해주세요.
    """,
    
    """
    이 데이터를 바탕으로 다음을 분석해주세요:
    1. 가장 성과가 좋았던 시기와 그 이유
    2. 개선이 필요한 영역
    3. 향후 3개월 예측 및 권장사항
    """,
    
    """
    경영진을 위한 요약 보고서를 작성해주세요. 
    핵심 성과 지표, 주요 인사이트, 실행 가능한 액션 아이템을 포함해서 
    간결하고 임팩트 있게 작성해주세요.
    """
]

print("=" * 70)
print("📊 GPT 5 비즈니스 차트 분석 결과")
print("=" * 70)

for i, prompt in enumerate(business_prompts, 1):
    print(f"\n📈 분석 {i}: 비즈니스 인사이트 추출")
    print("-" * 60)
    
    # GPT 4V로 차트 분석
    result = analyze_image_with_gpt5(chart_image_path, prompt, max_tokens=800)
    
    print(f"💼 분석 결과:\n{result}")
    print("-" * 60)


### 이미지를 활용해 분석해볼만한 것

1. OCR (텍스트 추출)
   - 문서, 영수증, 명함에서 텍스트 추출
   - 손글씨 인식 및 디지털화

2. 제품 분석
   - 제품 이미지 분석 및 설명 생성
   - 경쟁사 제품 비교 분석

3. 의료 이미지 분석
   - X-ray, MRI 등 의료 이미지 보조 분석
   - 피부 질환 이미지 분석

4. 부동산 / 건축
   - 건물 외관 분석 및 평가
   - 인테리어 디자인 제안

5. 교육 콘텐츠
   - 수학 문제 해결
   - 과학 실험 결과 분석


당연하게도 프롬프트는 명확하고 구체적으로 작성해야 답변의 질이 높아짐.  
이미지 분석의 경우에는 이미지 품질도 큰 비중을 차지함.  

단, 결과를 맹신하지 말 것  
개인정보가 포함된 이미지는 업로드하지 말 것  
